<a href="https://colab.research.google.com/github/JasmineVan/Page_Successor_Out-degree_Dead-ends/blob/main/ThuongTV_Presentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Big Data Processing**\
Thuong Trinh\
51800128


---



#1. Installation:

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Presentation').getOrCreate()

# Creates Empty RDD
emptyRDD = spark.sparkContext.emptyRDD()
print(emptyRDD)

EmptyRDD[0] at emptyRDD at NativeMethodAccessorImpl.java:0


In [57]:
from pyspark.sql.types import StructType,StructField, StringType
# Declare 2 Columns Page and Successor
schema = StructType([
  StructField('Page', StringType(), True),
  StructField('Successor', StringType(), True),
  StructField('Out-degree', StringType(), True),
  StructField('Dead-ends', StringType(), True)
  ])

In [58]:
# Create new Dataframe with empty RDD
df = spark.createDataFrame(emptyRDD,schema)
df.printSchema()

root
 |-- Page: string (nullable = true)
 |-- Successor: string (nullable = true)
 |-- Out-degree: string (nullable = true)
 |-- Dead-ends: string (nullable = true)



#2. Implementation:

In [9]:
from bs4 import BeautifulSoup
import urllib.request
import urllib.parse

In [59]:
url =  'https://tdtu.edu.vn/'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, 'html.parser')

In [66]:
def FindPageSuccessor(SourceURL, limit, dataframe):
  '''
  Function Objective:
  - Variables in function:
  # childs - childs url of current url
  # count - counting out-degree of current url
  # DomainName - the same domain in (a) and (b) 
  '''
  # Soup variables
  page = urllib.request.urlopen(SourceURL)
  soup = BeautifulSoup(page, 'html.parser')
  childs = []
  count = 0
  columns = ['page', 'successor', 'out-degree', 'dead-ends']
  DomainName = urllib.parse.urlsplit(SourceURL).hostname
  if dataframe.count() == limit:
    return dataframe
  else:
    for row in soup.find_all(['a', 'link']):
      url = str(row.get('href'))
      if 'http' and DomainName in url:
        childs.append(url)
        # Calculate out-degree and dead-ends of qualified url
        count = CalculateOutDegree(url)
        if dataframe.count() == limit:
          return dataframe
        else:
          temp = []
          temp.append(SourceURL)
          temp.append(url)
          temp.append(count)
          if count == 0:
            temp.append(1)
          else:
            temp.append(0)
          newRow = spark.createDataFrame([tuple(temp)], columns)
          dataframe = dataframe.union(newRow)
    for child in childs:
      # Recursive in current URL
      FindPageSuccessor(child, limit, dataframe)

In [67]:
def CalculateOutDegree(SourceURL):
  count = 0
  '''
  Function Objective:
  - Variables in function:
  # count - counting out-degree of current url
  '''
  # Soup variables
  page = urllib.request.urlopen(SourceURL)
  soup = BeautifulSoup(page, 'html.parser')
  for row in soup.find_all(['a', 'link']):
    url = str(row.get('href'))
    if 'http' in url:
      count += 1
  return count

In [71]:
limit = 20
df2 = FindPageSuccessor(url, limit, df)

In [72]:
df2.show(limit)

+--------------------+--------------------+----------+---------+
|                Page|           Successor|Out-degree|Dead-ends|
+--------------------+--------------------+----------+---------+
|https://tdtu.edu.vn/|https://tdtu.edu.vn/|        64|        0|
|https://tdtu.edu.vn/|https://tdtu.edu.vn/|        64|        0|
|https://tdtu.edu.vn/|https://tdtu.edu....|        64|        0|
|https://tdtu.edu.vn/|https://tdtu.edu.vn/|        64|        0|
|https://tdtu.edu.vn/|https://tdtu.edu....|        64|        0|
|https://tdtu.edu.vn/|https://college.t...|        15|        0|
|https://tdtu.edu.vn/|https://vfis.tdtu...|        35|        0|
|https://tdtu.edu.vn/|http://it.tdtu.ed...|        20|        0|
|https://tdtu.edu.vn/|http://pharmacy.t...|        12|        0|
|https://tdtu.edu.vn/|http://feee.tdtu....|         6|        0|
|https://tdtu.edu.vn/|http://aaf.tdtu.e...|        10|        0|
|https://tdtu.edu.vn/|http://fss.tdtu.e...|       186|        0|
|https://tdtu.edu.vn/|htt